## Objetivo do cralwer: Encontrar no site Airbnb, para uma determinada cidade, diversas informacoes a respeito de todas as acomodacoes atualmente disponíveis

## Cidade

In [1]:
# cidade desejada para buscar acomodações
cidade = 'Patos de Minas'

## Imports

In [2]:
import pandas as pd
import unidecode
import pickle
import time
from selenium import webdriver
from joblib import Parallel, delayed

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 20)

## Metodos

In [3]:
# trata o nome da cidade de acordo com o padrao esperado pelo site
def trata_nome_cidade(cidade):
    return (unidecode.unidecode(cidade.lower().replace(' ', '_')))

# formata a url de acordo com os parametros passados
def prepara_cidade_url(main_url, cidade, preco1, preco):
    return (main_url.format(cidade, preco1, preco2))

# retorna uma lista de faixas de preco
def get_faixas_precos(lista_precos):
    faixas_precos = []
    for i in range(len(lista_precos) - 1):
        faixas_precos.append([lista_precos[i], lista_precos[i+1]-1])
    return (faixas_precos)

# mostra o tempo de execucao decorrido
def print_tempo_execucao(start_time, end_time):
    print ('Tempo de execução: {} segundos'.format(round((end_time - start_time), 2)))

# recebe uma url de acomodacao e retorna suas informacos
def extrai_infos_urls(url_casa):
    # inicializa o driver
    driver = webdriver.Chrome(path_to_chromedriver, options=chromeOptions)

    driver.get(url_casa)
    time.sleep(0.5)
    
    # encontra informacao na pagina
    try:
        nome_lugar = driver.find_element_by_xpath(xpath_nome_lugar).text
    except:
        return (missing_values)

    # tenta achar a lat long
    for i in range(10):
        try:
            url_lat_long = driver.find_element_by_xpath(xpath_latlong).get_attribute("src")
            break
        except:
            time.sleep(2)

    try:
        lat_long = url_lat_long.split("&scale")[0].split("center=")[1]
    except:
        try:
            lat_long = url_lat_long.split("&client")[0].split("png|")[1]
        except:
            try:
                lat_long = url_lat_long.split("&amp")[0].split("center=")[1]
            except:
                lat_long = ''

    if lat_long != '':
        lat = lat_long.split(",")[0].replace('.', ',')
        long = lat_long.split(",")[1].replace('.', ',')

    # preço da diaria    
    for i in range(10):
        try:
            preco_diaria = driver.find_element_by_xpath(xpath_preco_diaria1).text
            break
        except:
            try:
                preco_diaria = driver.find_element_by_xpath(xpath_preco_diaria2).text
                break
            except:
                time.sleep(2)
                
    preco_diaria = str(preco_diaria).split('<')[0]

    anfitriao_acomodacao = driver.find_element_by_xpath(xpath_anfitriao).text
    regiao_acomodacao = driver.find_elements_by_xpath(xpath_regiao)[0].text
    tipo = driver.find_element_by_xpath(xpath_tipo).text
    capacidade_casa = driver.find_elements_by_xpath(xpath_capacidade_casa)[1].text
    quartos_casa = driver.find_elements_by_xpath(xpath_quartos_casa)[2].text

    comentarios = driver.find_elements_by_xpath(xpath_comentarios)[0].text.split(" ")[0]

    try:
        comentarios = int(comentarios)
    except:
        comentarios = 0
        
    if (comentarios == 0):
        data = ''

    elif (comentarios <= 6):
        for i in range(10):
            try:
                data = driver.find_elements_by_xpath(xpath_data)[-1].text
                break
            except:
                time.sleep(1)

    else:
        try:
            ultima_pag_comentarios = driver.find_elements_by_xpath(xpath_ultima_pagina_comentarios)[-1]
            driver.execute_script("arguments[0].click();", ultima_pag_comentarios)
            time.sleep(1)
        except:
            pass

        for i in range(10):
            try:
                data = driver.find_elements_by_xpath(xpath_data)[-1].text
                break
            except:
                time.sleep(1)

    # fecha o driver    
    driver.quit()
        
    # retorna resultados
    return ([cidade, anfitriao_acomodacao, url_casa, nome_lugar,
             regiao_acomodacao, tipo, capacidade_casa, quartos_casa,
             lat, long, preco_diaria, comentarios, data])

## Parametros globais

In [4]:
# url principal para pesquisar acomodacoes
main_url = 'https://www.airbnb.com.br/s/{0}/homes?refinement_paths%5B%5D=%2Fhomes&search_type=filter_change&price_min={1}&price_max={2}'

# condicao de parada do crawler
condicao_parada = 'items_offset=288'

# xpaths com os itens desejados
xpath_elements_pagina = '//a[contains(@href, "/rooms")]'
xpath_botao_muda_pagina = '//div[@class="_1m76pmy"]'
xpath_latlong = '//div[@class="_59m2yxn"]/img'
xpath_ultima_pagina_comentarios = '//div[@class="_1bdke5s"]'
xpath_data = '//span[@class="_1jlnvra2"]'
xpath_preco_diaria1 = '//span[@class="_doc79r"]'
xpath_preco_diaria2 = '//span[contains(text(), "R$")]'
xpath_nome_lugar = '//span[@class="_18hrqvin"]'
xpath_comentarios = '//span[@class="_s1tlw0m"]'
xpath_quartos_casa = '//div[@class="_czm8crp"]'
xpath_capacidade_casa = '//div[@class="_czm8crp"]'
xpath_tipo = '//div[@class="_1p3joamp"]'
xpath_regiao = '//div[@class="_czm8crp"]'
xpath_anfitriao = '//div[@class="_8b6uza1"]'

# faixas de preco das acomodacoes
# obs: metodo utilizado para maximizar o numero de acomodacoes encontradas
lista_precos = [1, 50, 60, 70, 75, 80, 85, 90, 100, 101, 105, 110,
                115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165,
                170, 175, 180, 185, 190, 195, 200, 210, 220, 230, 240,
                250, 260, 270, 280, 290, 300, 350, 400, 450, 500, 600,
                700, 800, 900, 1000, 2000, 100000]

# numero de navegadores utilizados em paralelo
num_browsers = 8

# path onde foi instalado o chromedriver
path_to_chromedriver = '/home/yan/Documentos/chromedriver'

missing_values = ['', '', '', '',
                 '', '', '', '',
                 '', '', '', '', '']

## Setup

In [5]:
# trata nome da cidade para colocar no formado a ser colocado na url
cidade_tratada = trata_nome_cidade(cidade)

# determina faixas de precos (lista de valores min e max) de cada busca
faixas_precos = get_faixas_precos(lista_precos)

# parametros do webdriver
chromeOptions = webdriver.ChromeOptions()
chromeOptions.add_argument("--headless")

## Parte 1) Acha todas as acomodacoes da cidade

In [6]:
start_time = time.time()

# inicializa o driver
driver = webdriver.Chrome(path_to_chromedriver, options=chromeOptions)

lista_url_casas = []

for preco1, preco2 in faixas_precos:

    flag_parada = 0
    lista_urls_visitadas = []

    # prepara a url com base no nome da cidade
    url_atual = prepara_cidade_url(main_url, cidade_tratada, preco1, preco2)

    driver.get(url_atual)

    while(flag_parada == 0):
        time.sleep(0.5)
        # caso seja a ultima pagina da cidade
        url_atual = driver.current_url

        # caso ja tenha visitado essa pagina
        if (url_atual in lista_urls_visitadas):
            flag_parada = 1
            break

        # caso seja a ultima pagina
        if (condicao_parada in url_atual):
            flag_parada = 1

        # atualiza lista de urls visitadas
        lista_urls_visitadas.append(url_atual)

        # lista elementos da pagina
        time.sleep(2)
        list_elements = driver.find_elements_by_xpath(xpath_elements_pagina)
        time.sleep(3)


        # coloca na lista acomodacoes encontradas
        for element in list_elements:
            try:
                url_casa = element.get_attribute('href')
                if (url_casa not in lista_url_casas):
                    lista_url_casas.append(url_casa)
            except Exception as e:
                continue

        # muda de pagina
        if (flag_parada == 0):
            try:
                botao = driver.find_elements_by_xpath(xpath_botao_muda_pagina)[-1]
                driver.execute_script("arguments[0].click();", botao)
            except:
                pass

# fecha o driver    
driver.quit()

print ('Numero encontrado de casas:', len(lista_url_casas))

# salva resultado em um arquivo pickle
with open('{}.pkl'.format(cidade_tratada), 'wb') as handle:
    pickle.dump(lista_url_casas, handle, protocol=pickle.HIGHEST_PROTOCOL)

# tempo de execucao
print_tempo_execucao(start_time, time.time())

Numero encontrado de casas: 30
Tempo de execução: 481.76 segundos


## Parte 2) Encontra informacoes das acomodacoes

In [7]:
start_time = time.time()

# leitura da lista de urls de acomodacoes encontradas
with open('{}.pkl'.format(cidade_tratada), 'rb') as handle:
    lista_url_casas = pickle.load(handle)

# extrai resultados das url em paralelo
resultados = Parallel(n_jobs=num_browsers)(delayed(extrai_infos_urls)(url) for url in lista_url_casas)

# salva tudo em unico dataframe
df_output = pd.DataFrame(resultados, columns=['cidades', 'anfitriao', 'url_acomodacao', 'nome_acomodacao', 'regiao',
'tipo_acomodacao','capacidade', 'quartos', 'latitude', 'longitude',
'precos','num_comentarios','data_primeiro_comentario'])

# tempo de execucao
print_tempo_execucao(start_time, time.time())

Tempo de execução: 81.88 segundos


## Salva resultado

In [8]:
# salva resultado em um arquivo xlsx
df_output.to_excel('resultados_{}.xlsx'.format(cidade_tratada), index=False)

## Resultado

In [9]:
df_output

,cidades,anfitriao,url_acomodacao,nome_acomodacao,regiao,tipo_acomodacao,capacidade,quartos,latitude,longitude,precos,num_comentarios,data_primeiro_comentario
0,Patos de Minas,Bethania,https://www.airbnb.com.br/rooms/25267072?locat...,Quarto privativo no Centro,Patos de Minas,Quarto inteiro em apartamento,3 hóspedes,1 quarto,"-18,58535","-46,51496",R$45,6,Julho de 2018
1,Patos de Minas,Rodrigo,https://www.airbnb.com.br/rooms/36024686?locat...,Recanto caseiro para um bom descanso.,Valparaíso,Quarto inteiro em casa,1 hóspede,1 quarto,"-18,58254","-46,50124",R$40,0,
2,Patos de Minas,Bethania,https://www.airbnb.com.br/rooms/24700722?locat...,Quarto Privativo em casa de família,São Francisco,Quarto inteiro em casa,2 hóspedes,1 quarto,"-18,5828","-46,51225",R$38,6,Setembro de 2018
3,Patos de Minas,Ermeson,https://www.airbnb.com.br/rooms/34805635?locat...,Quarto para fenamilho,Centro,Quarto inteiro em apartamento,2 hóspedes,1 quarto,"-18,59067","-46,50818",R$50,0,
4,Patos de Minas,Bethania,https://www.airbnb.com.br/rooms/33273006?locat...,Quarto em casa de família com garagem,Patos de minas,Quarto inteiro em casa,3 hóspedes,1 quarto,"-18,58133","-46,51351",R$43,2,Março de 2019
5,Patos de Minas,Maria Aparecida,https://www.airbnb.com.br/rooms/35663390?locat...,Hotel Point center,Guimarânia,Quarto em hotel,1 hóspede,1 quarto,"-18,84272","-46,79492",R$40,0,
6,Patos de Minas,Delmira E Jacques,https://www.airbnb.com.br/rooms/28306571?locat...,Hostel.ss quarto 1.1,Serra do Salitre,Quarto compartilhado em casa de hóspedes,2 hóspedes,1 quarto,"-19,11127","-46,68449",R$49,1,Fevereiro de 2019
7,Patos de Minas,Gabriel,https://www.airbnb.com.br/rooms/33855208?locat...,hospede se na casa da tia Ro,São Francisco,Quarto inteiro em casa,1 hóspede,1 quarto,"-18,58602","-46,51075",R$40,1,Abril de 2019
8,Patos de Minas,Delmira E Jacques,https://www.airbnb.com.br/rooms/27893679?locat...,Hostel.ss quarto 2,Serra do Salitre,Quarto compartilhado em casa de hóspedes,1 hóspede,1 quarto,"-19,11069","-46,68429",R$49,0,
9,Patos de Minas,Delmira E Jacques,https://www.airbnb.com.br/rooms/28509273?locat...,Hostel.ss quarto 3,Serra do Salitre,Quarto compartilhado em casa de hóspedes,3 hóspedes,1 quarto,"-19,10991","-46,68456",R$49,0,
